In [12]:
# ## inputs
water_xyz = water = """3

H\t0.7493682\t0.0\t0.4424329
O\t0.0\t0.0\t-0.1653507
H\t-0.7493682\t0.0\t0.4424329
#     """
# geometry = water_xyz[3:]

# methane = [
# ['C', (0.0000, 0.0000, 0.0000)],
# ['H', (0.5288, 0.1610, 0.9359)],
# ['H', (0.2051, 0.8240, -0.6786)],
# ['H', (0.3345, -0.9314, -0.4496)],
# ['H', (-1.0685, -0.0537, 0.1921)]
# ]

geometry = methane

In [13]:
from nbed.embed import nbed_driver


n_active_atoms=2
basis = 'STO-3G'
xc_functional = 'B3LYP'# 'lda, vwn' #'B3LYP'
convergence = 1e-6
loc_method = "ibo" #"spade" #"pipekmezey" "ibo" "boys"
proj_method = 'huzinaga' # 'huzinaga' #"mu_shift"
mu_level_shift=1e6
run_ccsd = True
run_fci = True
max_ram = 8_000

output = None

# q_ham, classical_energy = nbed_driver(
#     geometry ,
#     n_active_atoms,
#     basis,
#     xc_functional,
#     output,
#     convergence,
#     localization_method= loc_method,
#     projector_method = proj_method,
#     mu_level_shift = mu_level_shift,
#     run_ccsd_emb = run_ccsd,
#     max_ram_memory = max_ram,
#     run_fci_emb=run_fci)

outdic = nbed_driver(
    geometry ,
    n_active_atoms,
    basis,
    xc_functional,
    output,
    convergence,
    localization_method= loc_method,
    projector_method = proj_method,
    mu_level_shift = mu_level_shift,
    run_ccsd_emb = run_ccsd,
    max_ram_memory = max_ram,
    run_fci_emb=run_fci,
    run_global_fci=True)


 Iterative localization: IB/P4/2x2, 6 iter; Final gradient 5.56e-10
embedded HF energy MU_SHIFT: -16.08934263407614, converged: True
CCSD Energy:
	%s -39.652659217310806
FCI MU Energy:
	%s -39.85904849172825
embedded HF energy HUZINAGA: -16.089342579894534, converged: True
CCSD Energy:
	%s -39.65265916285462
FCI HUZ Energy:
	%s -39.85904839550517
difference between CCSD Huz and Mu calcs: 5.4456187115192733e-08
difference between FCI Huz and Mu calcs: 9.62230828349675e-08
core constant: 0
core constant: 0
num e emb: 4
[0 2]
[1 3 4]
global FCI: -39.805676932960054


In [ ]:
-16.514892640009133 + 16.514892561936392

In [8]:
qubit_H_mu = outdic['mu_shift']['H']
qubit_H_huz = outdic['huzinaga']['H']

from openfermion.linalg import get_sparse_operator
H_sparse_mu = get_sparse_operator(qubit_H_mu)
H_sparse_huz = get_sparse_operator(qubit_H_huz)

In [9]:
import scipy as sp
eig_vals_emb_mu, eig_vecs_emb_mu = sp.sparse.linalg.eigsh(H_sparse_mu,
                                                   which='SA',
                                                   k=1)
del H_sparse_mu

eig_vals_emb_huz, eig_vecs_emb_huz = sp.sparse.linalg.eigsh(H_sparse_huz,
                                                   which='SA',
                                                   k=1)
del H_sparse_huz

In [10]:
import numpy as np
mu_classical_E = outdic['mu_shift']['energy_classical']
huz_classical_E = outdic['huzinaga']['energy_classical']

mu_full_quantum = eig_vals_emb_mu[0] + mu_classical_E
huz_full_quantum = eig_vals_emb_huz[0] + huz_classical_E

print(mu_full_quantum)
print(huz_full_quantum)

print(np.abs(huz_full_quantum-mu_full_quantum))

-39.85904849139898
-39.859048395506434
9.589254545971926e-08


In [11]:
N_elec_qubit_state = np.binary_repr(
                                    np.where(np.abs(eig_vecs_emb_huz)>1e-2)[0][0]
                                    ).count('1')
N_elec_qubit_state

4

In [ ]:
-39.96169872936931
-39.961698797438004
6.806869379261116e-08

-39.909061846092804
-39.90906181385512
3.223768629823098e-08

-39.859048491530274
-39.85904839550501
9.602526063190453e-08



In [ ]:
eig_vals_emb_mu[0] + mu_classical_E

In [ ]:
from openfermion.linalg import get_sparse_operator
H_sparse = get_sparse_operator(q_ham)

In [ ]:
import scipy as sp
eig_vals_emb, eig_vecs_emb = sp.sparse.linalg.eigsh(H_sparse,
                                                   which='SA',
                                                   k=1)
del H_sparse

In [ ]:
E_VQE = eig_vals_emb[0] + classical_energy
E_VQE

In [ ]:
import numpy as np

N_elec_qubit_state = np.binary_repr(
                                    np.where(np.abs(eig_vecs_emb)>1e-2)[0][0]
                                    ).count('1')
N_elec_qubit_state

In [ ]:
act = [0,2,4]
env = [1,3]

[i for i in range(len(act), len(act) + len(env))]

In [ ]:
mu_CCSD = -75.86087423588484
mu_FCI =  -76.09254135408943
mu_VQE = -76.02674879647752

huz_CCSD = -75.85383455242645
huz_FCI =  -75.85383404201579
huz_VQE = -75.8538340123436

print(np.abs(mu_CCSD-huz_CCSD))
print(np.abs(mu_VQE-huz_VQE))
print(np.abs(mu_FCI-huz_FCI))

print(np.abs(mu_VQE-mu_FCI))
print(np.abs(huz_VQE-huz_FCI))